<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/PCA_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script to PCA analysis with Python**


In [1]:
# --- Intall MDAnalysis
!pip install MDAnalysis tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
# --- Import libraries ---
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tqdm import tqdm
import MDAnalysis as mda
from MDAnalysis.analysis import align

In [ ]:
# === Upload GROMACS files ===
print("Please upload md.tpr and traj.xtc")
uploaded = files.upload()

In [ ]:
# Load trajectory (must have md.tpr and traj.xtc)
u = mda.Universe("md.tpr", "traj.xtc")

In [ ]:
# Select C-alpha atoms
ca = u.select_atoms("protein and name CA")

In [ ]:
# Align trajectory to the first frame to remove overall translation/rotation
ref = u.select_atoms("protein and name CA")
aligner = align.AlignTraj(u, ref, select="protein and name CA", in_memory=True).run()

In [ ]:
# ===== Build coordinate matrix =====
print("Building coordinate matrix...")

coords = []
for ts in tqdm(u.trajectory, desc="Processing frames"):
    coords.append(protein.positions.flatten())

X = np.array(coords)
X -= X.mean(axis=0)  # mean-center the data

In [ ]:
# ===== Covariance matrix =====
print("Computing covariance matrix...")
cov = np.cov(X.T)

In [ ]:
# === Eigen-decomposition ===
eigvals, eigvecs = np.linalg.eigh(cov)

In [ ]:
# Sort by descending eigenvalue
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

In [ ]:
# ===== Projections on PC1 and PC2 =====
proj = np.dot(X, eigvecs[:, :2])


In [ ]:
# ===== Plotting =====
print("Generating plots...")

# Scree plot
plt.figure(figsize=(8, 5))
plt.plot(np.arange(1, 11), eigvals[:10], "o-", lw=2)
plt.xlabel("Principal Component", fontsize=12)
plt.ylabel("Eigenvalue (variance)", fontsize=12)
plt.title("Scree Plot of PCA", fontsize=14)
plt.grid(True)
plt.tight_layout()
plt.savefig("PCA_scree.png", dpi=1200, bbox_inches="tight")
plt.close()

# PC1 vs PC2 plot
plt.figure(figsize=(7, 6))
plt.scatter(proj[:, 0], proj[:, 1], alpha=0.5, s=10, c=np.arange(len(proj)))
plt.xlabel("PC1", fontsize=12)
plt.ylabel("PC2", fontsize=12)
plt.title("PCA Projection (PC1 vs PC2)", fontsize=14)
plt.colorbar(label="Frame Index")
plt.grid(True)
plt.tight_layout()
plt.savefig("PCA_projection.png", dpi=1200, bbox_inches="tight")
plt.close()


In [ ]:
 ===== Download Figures =====
print("Saving and downloading figures...")
files.download("PCA_scree.png")
files.download("PCA_projection.png")

print("PCA analysis completed!")